In [70]:
# foi necessário baixar manualmente o pacote e substituir o que foi baixado pelo '!pip install redis'
import redis

In [71]:
redis = redis.Redis('localhost')

In [72]:
# cria números para cartelas
def cria_numeros():
    for x in range(99):
        redis.sadd("numeros", x+1)

In [73]:
# cadastra os 50 jogadores
ctx = 1
while ctx <= 50:
    i = str(ctx)
    redis.hset("jogador"+i, "nome", "jogador"+i)
    redis.hset("jogador"+i, "cartela", "cartela"+i)
    redis.hset("jogador"+i, "pontuacao", "pontuacao"+i)
    
    ctx += 1

In [74]:
# cria as cartelas as 50 cartelas para os 50 jogadores
for x in range(50):
    cria_numeros()
    for i in range(15):
        num = redis.srandmember('numeros')
        redis.sadd("cartela"+str(x+1), num)
        redis.srem('numeros', num)

In [75]:
# cria pontuação zero para todos os jogadores iniciarem o sorteio
for x in range(50):
    redis.zadd("pontuacao", {"pontuacao"+str(x+1): 0})

In [76]:
# efetua o sorteio até ao menos um jogador atingir 15 pontos
cria_numeros()
i = 0
while i != 1:  
    num = redis.srandmember('numeros')
    redis.srem('numeros', num)
    
    for x in range(50):
        ctl = 'cartela'+str(x+1)
        scr = 'pontuacao'+str(x+1)
        nm = 'jogador'+str(x+1)
        if redis.sismember(ctl,num):
            redis.zincrby('pontuacao', float(1), scr)
            
        if len(redis.zrangebyscore('pontuacao',15,15)) > 0:
            i = 1

In [77]:
# lista de jogadores que atingiram 15 pontos
ganhadores = redis.zrangebyscore('pontuacao',15,15)
print(redis.zrangebyscore('pontuacao',15,15))

[b'pontuacao33', b'pontuacao40', b'pontuacao44', b'pontuacao5', b'pontuacao6']


In [78]:
# lista dos nomes dos jogadores com 15 pontos
for x in range(50):
    for i in ganhadores:
        if redis.hget('jogador'+str(x+1), 'pontuacao') == i:
            print(redis.hget('jogador'+str(x+1), 'nome'))

b'jogador5'
b'jogador6'
b'jogador33'
b'jogador40'
b'jogador44'
